# load required modules

In [1]:
import sys, os, pygmt
sys.path.insert(0, '/home/581/da1339/AFIM/src/AFIM/src')
from fast_ice_processor import FastIceProcessor
from pack_ice_processor import PackIceProcessor
from sea_ice_plotter    import SeaIcePlotter
import numpy            as np
import pandas           as pd
import xarray           as xr
from datetime           import timedelta, date, datetime

# process for pack ice

In [2]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-12-31"
sim_name = 'ktens-max'
mask_fi  = True
PI_proc  = PackIceProcessor(sim_name      = sim_name,
                            dt0_str       = dt0_str,
                            dtN_str       = dtN_str,
                            mask_fast_ice = mask_fi)
PI       = PI_proc.process_window(save_zarr=True, ow_zarrs=True)

2025-04-01 14:21:58,760 - INFO - ✅ Model dataset loaded: shape Frozen({'time': 15, 'nj': 1080, 'ni': 1440}), time: 1.27 s
INFO:ktens-max_PI:✅ Model dataset loaded: shape Frozen({'time': 15, 'nj': 1080, 'ni': 1440}), time: 1.27 s
2025-04-01 14:21:58,764 - INFO - 🧭 Including NSIDC SIA/SIE into model dataset for ktens-max
INFO:ktens-max_PI:🧭 Including NSIDC SIA/SIE into model dataset for ktens-max
2025-04-01 14:21:59,083 - INFO - ✅ NSIDC dataset loaded: shape Frozen({'tdim': 15, 'y': 332, 'x': 316}), time: 0.32 s
INFO:ktens-max_PI:✅ NSIDC dataset loaded: shape Frozen({'tdim': 15, 'y': 332, 'x': 316}), time: 0.32 s
2025-04-01 14:21:59,087 - INFO - 🧭 Converting NSIDC Cartesian to spherical coordinates:
INFO:ktens-max_PI:🧭 Converting NSIDC Cartesian to spherical coordinates:
2025-04-01 14:21:59,154 - INFO - 	✅ Conversion complete in 0.07 seconds
INFO:ktens-max_PI:	✅ Conversion complete in 0.07 seconds
2025-04-01 14:21:59,226 - INFO - ✅ Computed SIA/SIE in 0.07 seconds
INFO:ktens-max_PI:✅ Com




 ['PI_FRAZIL', 'PI_STRONG', 'PI_DIV', 'PI_SPEED', 'PIE', 'SIA', 'PI_AGROM', 'PI_VGROT', 'PI_AGROT', 'PI_AGING', 'PI_SHEAR', 'PI_VGROM', 'SIE', 'PIV', 'PIA', 'PI_STRESS'] 





2025-04-01 14:22:04,660 - INFO - 	3D vars computed: ['PIHI', 'PIAT', 'PIC', 'PIAG', 'PIVT', 'PISPD', 'PIDIV', 'PIVD', 'PIAD', 'PISTR', 'PIFZL', 'PISH', 'PISTH']
INFO:ktens-max_PI:	3D vars computed: ['PIHI', 'PIAT', 'PIC', 'PIAG', 'PIVT', 'PISPD', 'PIDIV', 'PIVD', 'PIAD', 'PISTR', 'PIFZL', 'PISH', 'PISTH']
2025-04-01 14:22:04,662 - INFO - compute temporal means to give spatial distributions over time (2D)
INFO:ktens-max_PI:compute temporal means to give spatial distributions over time (2D)
2025-04-01 14:23:09,769 - INFO - 	2D vars computed: ['PIAT_SD', 'PIAD_SD', 'PISTR_SD', 'PIC_SD', 'PIFZL_SD', 'PIVT_SD', 'PIAG_SD', 'PISPD_SD', 'PISTH_SD', 'PISH_SD', 'PIDIV_SD', 'PIVD_SD', 'PIHI_SD']
INFO:ktens-max_PI:	2D vars computed: ['PIAT_SD', 'PIAD_SD', 'PISTR_SD', 'PIC_SD', 'PIFZL_SD', 'PIVT_SD', 'PIAG_SD', 'PISPD_SD', 'PISTH_SD', 'PISH_SD', 'PIDIV_SD', 'PIVD_SD', 'PIHI_SD']
2025-04-01 14:23:09,773 - INFO - create output dataset:
INFO:ktens-max_PI:create output dataset:
2025-04-01 14:23:09,895 




 ['PI_FRAZIL', 'PI_STRONG', 'PI_DIV', 'PI_SPEED', 'PIE', 'SIA', 'PI_AGROM', 'PI_VGROT', 'PI_AGROT', 'PI_AGING', 'PI_SHEAR', 'PI_VGROM', 'SIE', 'PIV', 'PIA', 'PI_STRESS'] 





2025-04-01 14:23:43,343 - INFO - 	3D vars computed: ['PIHI', 'PIAT', 'PIC', 'PIAG', 'PIVT', 'PISPD', 'PIDIV', 'PIVD', 'PIAD', 'PISTR', 'PIFZL', 'PISH', 'PISTH']
INFO:ktens-max_PI:	3D vars computed: ['PIHI', 'PIAT', 'PIC', 'PIAG', 'PIVT', 'PISPD', 'PIDIV', 'PIVD', 'PIAD', 'PISTR', 'PIFZL', 'PISH', 'PISTH']
2025-04-01 14:23:43,345 - INFO - compute temporal means to give spatial distributions over time (2D)
INFO:ktens-max_PI:compute temporal means to give spatial distributions over time (2D)


KeyboardInterrupt: 

# plot 1D analysis

In [2]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-12-31"
sim_name = 'ktens-max'
plotter  = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=True, ice_type='PI', mask_fast_ice=True)
plotter.plot_timeseries()

No Zarr files found in /g/data/gv90/da1339/afim_output/ktens-max/PI
returning empty list
⏳ No Zarr time series found, generating for vars: ['PIA', 'SIA', 'PIE', 'SIE']


2025-04-01 09:40:43,536 - INFO - ✅ Model dataset loaded: shape Frozen({'time': 365, 'nj': 1080, 'ni': 1440}), time: 37.27 s
INFO:ktens-max_PI:✅ Model dataset loaded: shape Frozen({'time': 365, 'nj': 1080, 'ni': 1440}), time: 37.27 s
2025-04-01 09:40:43,540 - INFO - 🧭 Including NSIDC SIA/SIE into model dataset for ktens-max
INFO:ktens-max_PI:🧭 Including NSIDC SIA/SIE into model dataset for ktens-max
2025-04-01 09:41:14,212 - INFO - ✅ NSIDC dataset loaded: shape Frozen({'tdim': 365, 'y': 332, 'x': 316}), time: 30.66 s
INFO:ktens-max_PI:✅ NSIDC dataset loaded: shape Frozen({'tdim': 365, 'y': 332, 'x': 316}), time: 30.66 s
2025-04-01 09:41:14,216 - INFO - 🧭 Converting NSIDC Cartesian to spherical coordinates:
INFO:ktens-max_PI:🧭 Converting NSIDC Cartesian to spherical coordinates:
2025-04-01 09:41:14,283 - INFO - 	✅ Conversion complete in 0.07 seconds
INFO:ktens-max_PI:	✅ Conversion complete in 0.07 seconds
2025-04-01 09:41:15,836 - INFO - ✅ Computed SIA/SIE in 1.55 seconds
INFO:ktens-max_

KeyboardInterrupt: 

In [ ]:
#plotter.plot_timeseries(var_names=["PI_STRESS"])
plotter.plot_timeseries_compare(
    comparison_name = "tensile-stars",
    sim_names       = ["baseline", "ktens-max", "Pstar-max", "Cstar-min"],
    var_name        = "PIA",
    label_dict      = {"baseline"  : "tensile = 0.2, Pstar = 2.75e4, Cstar = 20",
                       "ktens-max" : "tensile = 0.3, Pstar = 2.75e4, Cstar = 20",
                       "Pstar-max" : "tensile = 0.2, Pstar = 5e4   , Cstar = 20",
                       "Cstar-min" : "tensile = 0.2, Pstar = 2.75e4, Cstar = 10"})

# plot 2D analysis (requires existing FI/PI dataset)

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1999-12-31"
sim_names = ["baseline", "ustar-low", "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min",
             "ktens-nil", "ktens-min", "ktens-max", "gi-max", "gi-min",
             "Rothrock", "elps-pack", "elps-fast", "elps-max", "elps-def", "elps-mid" ]
for sim_name in sim_names:
    SI_plot = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=False, overwrite=True, single_figure=False)
    SI_plot.prepare_data_for_plotting(var_name='FIP')
    SI_plot.plot_regions()

# generate PBS jobs for analysis

In [3]:
from pathlib import Path

sim_names = ["Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min", "Rothrock", "baseline",
             "elps-def", "elps-fast", "elps-max", "elps-mid", "elps-pack", "gi-max",
             "gi-min", "ktens-max", "ktens-min", "ktens-nil", "oras-sfc-ctl", "ustar-low", "van_achter"]
pbs_template = """#!/bin/bash
#PBS -N PI-proc-{name}
#PBS -P gv90
#PBS -l walltime=24:00:00
#PBS -q normalbw
#PBS -l mem=32GB
#PBS -l ncpus=28
#PBS -l storage=gdata/cj50+gdata/jk72+scratch/jk72+gdata/ik11+gdata/hh5+gdata/rt52+gdata/gb6+gdata/gv90
#PBS -M daniel.atwater@utas.edu.au

module purge
module use /g/data/hh5/public/modules
module load conda/analysis3

cd ~/AFIM/src/AFIM/scripts
python3 ./compute_pack_ice.py {name}
"""

output_dir = Path("/home/581/da1339/AFIM/src/sh/analysis")
output_dir.mkdir(exist_ok=True)

for sim in sim_names:
    script_content = pbs_template.format(name=sim)
    script_path = Path(output_dir,f"PI-proc_{sim}.pbs")
    with open(script_path, "w") as f:
        f.write(script_content)
print(f"✓ Generated {len(sim_names)} PBS scripts in: {output_dir.resolve()}")

✓ Generated 19 PBS scripts in: /home/581/da1339/AFIM/src/sh/analysis
